In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [4]:
simpleData = (("James", "Sales", 3000), \
              ("Michael", "Sales", 4600),  \
              ("Robert", "Sales", 4100),   \
              ("Maria", "Finance", 3000),  \
              ("James", "Sales", 3000),    \
              ("Scott", "Finance", 3300),  \
              ("Jen", "Finance", 3900),    \
              ("Jeff", "Marketing", 3000), \
              ("Kumar", "Marketing", 2000),\
              ("Saif", "Sales", 4100) \
             )

In [5]:
columns= ["employee_name", "department", "salary"]

In [6]:
df = spark.createDataFrame(data = simpleData, schema = columns)

In [7]:
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [8]:
df.show(truncate=False)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+

